In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.vgg16 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

seed = 7

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(seed)

In [3]:
data_dir = '/data/oxford102/train'
img_width, img_height = 256, 256
batch_size = 32
epochs = 100
nr_categories = 102

In [4]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [5]:
input_processor = keras.applications.vgg16.preprocess_input

In [6]:
train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)

In [7]:

    
train_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')
validation_generator = train_val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.


In [14]:
k.set_learning_phase(0)

network_name = "vgg16"
img_width, img_height = (256, 256)
if network_name == "vgg16":
    base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
elif network_name == "vgg19":
    base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
else:
    raise Exception("check your network name")

for layer in base_model.layers[:]:
    layer.trainable = False

    #Adding custom Layers 
k.set_learning_phase(1)
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x, training=True)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

_model = Model(input = base_model.input, output = predictions)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [15]:
_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [10]:
_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [17]:
#run_training(model_name, _model, train_generator, validation_generator, params, num_train_img, num_val_img):
    

model_name = "vgg16"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
35/35 [==============================] - 18s 505ms/step - loss: 4.5033 - acc: 0.1063 - val_loss: 3.5819 - val_acc: 0.2383

Epoch 00001: val_acc improved from -inf to 0.23828, saving model to /data/oxford102/experiments/1542069202.4178612/vgg16_1542069202.4178612.h5
Epoch 2/100
35/35 [==============================] - 17s 474ms/step - loss: 3.0453 - acc: 0.3232 - val_loss: 2.6389 - val_acc: 0.4062

Epoch 00002: val_acc improved from 0.23828 to 0.40625, saving model to /data/oxford102/experiments/1542069202.4178612/vgg16_1542069202.4178612.h5
Epoch 3/100
35/35 [==============================] - 17s 477ms/step - loss: 2.4431 - acc: 0.4427 - val_loss: 2.2150 - val_acc: 0.4727

Epoch 00003: val_acc improved from 0.40625 to 0.47266, saving model to /data/oxford102/experiments/1542069202.4178612/vgg16_1542069202.4178612.h5
Epoch 4/100
35/35 [==============================] - 17s 474ms/step - loss: 2.0987 - acc: 0.5152 - val_loss: 1.8770 - val_acc: 0.5898

Epoch 00004: val_acc impr

In [18]:
model_name = "vgg16"
num_train_img = 4604
num_val_img = 1094

params = {'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 64,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
71/71 [==============================] - 35s 489ms/step - loss: 0.1904 - acc: 0.9911 - val_loss: 1.1201 - val_acc: 0.7261

Epoch 00001: val_acc improved from -inf to 0.72610, saving model to /data/oxford102/experiments/1542069948.9042153/vgg16_1542069948.9042153.h5
Epoch 2/100
71/71 [==============================] - 33s 468ms/step - loss: 0.1699 - acc: 0.9921 - val_loss: 1.0966 - val_acc: 0.7500

Epoch 00002: val_acc improved from 0.72610 to 0.75000, saving model to /data/oxford102/experiments/1542069948.9042153/vgg16_1542069948.9042153.h5
Epoch 3/100
71/71 [==============================] - 35s 487ms/step - loss: 0.1297 - acc: 0.9965 - val_loss: 1.0767 - val_acc: 0.7390

Epoch 00003: val_acc did not improve from 0.75000
Epoch 4/100
71/71 [==============================] - 34s 473ms/step - loss: 0.1251 - acc: 0.9960 - val_loss: 1.0593 - val_acc: 0.7702

Epoch 00004: val_acc improved from 0.75000 to 0.77022, saving model to /data/oxford102/experiments/1542069948.9042153/vgg

In [19]:
model_name = "vgg16"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 32,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
143/143 [==============================] - 69s 485ms/step - loss: 0.0780 - acc: 0.9978 - val_loss: 1.0344 - val_acc: 0.7629

Epoch 00001: val_acc improved from -inf to 0.76287, saving model to /data/oxford102/experiments/1542070195.6867728/vgg16_1542070195.6867728.h5
Epoch 2/100
143/143 [==============================] - 70s 488ms/step - loss: 0.0669 - acc: 0.9987 - val_loss: 1.0309 - val_acc: 0.7555

Epoch 00002: val_acc did not improve from 0.76287
Epoch 3/100
143/143 [==============================] - 69s 483ms/step - loss: 0.0578 - acc: 0.9998 - val_loss: 1.0022 - val_acc: 0.7739

Epoch 00003: val_acc improved from 0.76287 to 0.77390, saving model to /data/oxford102/experiments/1542070195.6867728/vgg16_1542070195.6867728.h5
Epoch 4/100
143/143 [==============================] - 69s 481ms/step - loss: 0.0512 - acc: 0.9998 - val_loss: 1.0075 - val_acc: 0.7629

Epoch 00004: val_acc did not improve from 0.77390
Epoch 5/100
143/143 [==============================] - 69s 483m

In [20]:
model_name = "vgg16"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
287/287 [==============================] - 139s 483ms/step - loss: 0.0335 - acc: 0.9999 - val_loss: 1.0041 - val_acc: 0.7572

Epoch 00001: val_acc improved from -inf to 0.75721, saving model to /data/oxford102/experiments/1542070615.581809/vgg16_1542070615.581809.h5
Epoch 2/100
287/287 [==============================] - 139s 484ms/step - loss: 0.0291 - acc: 1.0000 - val_loss: 0.9644 - val_acc: 0.7749

Epoch 00002: val_acc improved from 0.75721 to 0.77488, saving model to /data/oxford102/experiments/1542070615.581809/vgg16_1542070615.581809.h5
Epoch 3/100
287/287 [==============================] - 139s 484ms/step - loss: 0.0255 - acc: 1.0000 - val_loss: 0.9489 - val_acc: 0.7674

Epoch 00003: val_acc did not improve from 0.77488
Epoch 4/100
287/287 [==============================] - 138s 482ms/step - loss: 0.0225 - acc: 1.0000 - val_loss: 0.9373 - val_acc: 0.7795

Epoch 00004: val_acc improved from 0.77488 to 0.77953, saving model to /data/oxford102/experiments/1542070615.581

# FINETUNE TOP LAYERS

In [25]:
for i, layer in enumerate(base_model.layers[11:]):
    print(i, layer.name)
    layer.trainable = True

0 block4_conv1
1 block4_conv2
2 block4_conv3
3 block4_pool
4 block5_conv1
5 block5_conv2
6 block5_conv3
7 block5_pool


In [26]:
model_name = "vgg16"
num_train_img = 4604
num_val_img = 1094
params = {'network_name': 'vgg16',
         'image_aug': False,
         'optimizer': 'SGD',
          'optimizer_params': None, 
         'batch_size': 16,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}


np.random.seed(seed)
log_time = time()
params['log_time'] = log_time
batch_size = params.get("batch_size")

_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.001, momentum=0.9), metrics=["accuracy"])


base = '/data/oxford102/experiments'
path = os.path.join(base, str(log_time))
checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

try:
    if not os.path.exists(path):
        os.makedirs(path)
    history_callback = _model.fit_generator(
            train_generator,
            steps_per_epoch=num_train_img // batch_size,
            epochs=params.get("epochs"),
            validation_data=validation_generator,
            validation_steps=num_val_img // batch_size,
            callbacks = [checkpoint, early, tensorboard, csv_logger])
except Exception as e:
    raise(e)
finally:
    pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
    _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
    print(params)
    params

Epoch 1/100
287/287 [==============================] - 181s 632ms/step - loss: 0.0071 - acc: 1.0000 - val_loss: 0.8273 - val_acc: 0.8000

Epoch 00001: val_acc improved from -inf to 0.80000, saving model to /data/oxford102/experiments/1542073918.6993766/vgg16_1542073918.6993766.h5
Epoch 2/100
287/287 [==============================] - 178s 621ms/step - loss: 0.0059 - acc: 1.0000 - val_loss: 0.8125 - val_acc: 0.7972

Epoch 00002: val_acc did not improve from 0.80000
Epoch 3/100
287/287 [==============================] - 178s 621ms/step - loss: 0.0073 - acc: 1.0000 - val_loss: 0.7900 - val_acc: 0.8130

Epoch 00003: val_acc improved from 0.80000 to 0.81302, saving model to /data/oxford102/experiments/1542073918.6993766/vgg16_1542073918.6993766.h5
Epoch 4/100
287/287 [==============================] - 178s 621ms/step - loss: 0.0062 - acc: 1.0000 - val_loss: 0.7772 - val_acc: 0.8079

Epoch 00004: val_acc did not improve from 0.81302
Epoch 5/100
287/287 [==============================] - 178s